In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
settings = load_settings('binance_futures', 'your_user_name')
s = 'BTCUSDT'
n_days = 7

In [ ]:
agg_trades = await load_trades(s, n_days)

In [ ]:
agg_trades.iloc[::200].price.plot()

In [ ]:
adf = agg_trades

In [ ]:
# modify settings
settings = {
    'ema_spans': [3111, 4122],
    'markup': 0.00183,
    'leverage': 99,
    'spread': -0.00054,
    'entry_amount': 0.001,
    'enter_long': True,
    'enter_shrt': True
}


In [ ]:
trades, adf = backtest(adf, settings)

In [ ]:
tdf = pd.DataFrame(trades).set_index('trade_id')
bdf = tdf[tdf.side == 'buy']
sdf = tdf[tdf.side == 'sel']
tdf

In [ ]:
min_name = f"min_" + '_'.join([f'{int(span)}' for span in settings['ema_spans']])
max_name = f"max_" + '_'.join([f'{int(span)}' for span in settings['ema_spans']])
nx = 100000
adf[[min_name, max_name]].iloc[-nx::100].plot()
adf.price.iloc[-nx::100].plot()

In [ ]:
realized_pnl_sum = tdf.realized_pnl.sum()
fee_sum = tdf.fee.sum()
print('realized pnl sum', realized_pnl_sum)
print('fee sum', fee_sum)
print('pnl - fee', realized_pnl_sum - fee_sum)
print('pct fees paid', fee_sum / realized_pnl_sum)

In [ ]:
print('pos amount counts')
tdf.amount.value_counts()

In [ ]:
tdf.n_double_downs.value_counts()

In [ ]:
min_needed_margin = tdf.margin_cost.max()
print('min needed margin', min_needed_margin)

In [ ]:
(tdf.realized_pnl - tdf.fee).cumsum().plot()

In [ ]:
tdf.amount.plot()

In [ ]:
gain = (realized_pnl_sum - fee_sum + min_needed_margin) / min_needed_margin
n_days = (tdf.timestamp.max() - tdf.timestamp.min()) / 1000 / 60 / 60 / 24
adg = gain ** (1 / n_days)
average_daily_abs_profit = realized_pnl_sum / n_days
print('min_needed_margin', min_needed_margin)
print('realized_pnl_sum', realized_pnl_sum)
print('fee_sum', fee_sum)
print('net_abs_gain', realized_pnl_sum - fee_sum)
print('gain', gain)
print('n_days', n_days)
print('average daily gain', adg)
print('average_daily_abs_profit', average_daily_abs_profit)

In [ ]:
# visualize highest double downs

In [ ]:
idx = tdf.amount.abs().sort_values().index[-1]
idxloc = tdf.index.get_loc(idx)
n_dd = tdf.n_double_downs.iloc[idxloc-1]
istart = tdf.iloc[idxloc-n_dd - 50].name
iend = tdf.iloc[idxloc + 50].name
print(istart, iend, n_dd)
tdf.iloc[idxloc-n_dd-4:idxloc+2]

In [ ]:
min_name = f"min_" + '_'.join([f'{int(span)}' for span in settings['ema_spans']])
max_name = f"max_" + '_'.join([f'{int(span)}' for span in settings['ema_spans']])
adf.loc[istart:iend][['price', min_name, max_name]].plot(style='y')
bdfc = bdf[(bdf.index >= istart) & (bdf.index < iend)]
sdfc = sdf[(sdf.index >= istart) & (sdf.index < iend)]

ledfc = bdfc[bdfc.type == 'entry']
lxdfc = sdfc[sdfc.type == 'exit']

sedfc = sdfc[sdfc.type == 'entry']
sxdfc = bdfc[bdfc.type == 'exit']

lxdfc.price.plot(style='r<')
sxdfc.price.plot(style='b<')
ledfc.price.plot(style='b>')
sedfc.price.plot(style='r>')

In [ ]:
# for testing multiple settings

In [ ]:
results = jackrabbit(agg_trades)

In [ ]:
rdf = pd.DataFrame(results).T
rdf.sort_values('gain', ascending=[False])